In [44]:
import requests

In [45]:
from bs4 import BeautifulSoup
import geopandas as gpd
import pandas as pd

## Open State data

In [46]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

## Open Population Data

In [47]:
state_populations = pd.read_excel(
    "data/NST-EST2024-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [48]:
state_populations_df = state_populations["NST-EST2024-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [49]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "NAME", "POPULATION", "geometry"]
]

## Get Corn Data

In [55]:
corn_df = pd.read_csv("data/C99EEF3B-BA7C-3EB4-90E2-E57249E7B4A4.csv", thousands=",")
corn_df = corn_df.rename(columns={"State": "NAME", "Value": "AMOUNT"})
corn_df["NAME"] = corn_df["NAME"].str.capitalize()
corn_df = corn_df[["NAME", "AMOUNT"]]

## Merge Data

In [56]:
corn_gdf = states_with_population_df.merge(corn_df, on="NAME", how="left").fillna(0)

In [57]:
corn_gdf["per_100"] = corn_gdf["AMOUNT"] / (corn_gdf["POPULATION"] / 100)
corn_gdf["per_1000"] = corn_gdf["AMOUNT"] / (corn_gdf["POPULATION"] / 1_000)
corn_gdf["per_100k"] = corn_gdf["AMOUNT"] / (corn_gdf["POPULATION"] / 100_000)
corn_gdf["per_1m"] = corn_gdf["AMOUNT"] / (corn_gdf["POPULATION"] / 1_000_000)

In [58]:
corn_gdf = corn_gdf.to_crs(9311)
corn_gdf.to_file("data/corn_per_state.gpkg")